In [3]:
import os
import glob
import wave
import numpy as np
from scipy.signal import butter, lfilter
import soundfile as sf

# Function to apply a Butterworth band-pass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Parameters for digitization
lowcut = 300.0  # Lower cutoff frequency for pre-filtering (Hz)
highcut = 3400.0  # Upper cutoff frequency for pre-filtering (Hz)
fs = 8000  # Sampling rate (Hz)
n_bits = 16  # Quantization accuracy (bits)

# Path to the directory containing folders with .wav files
root_path =  "d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data"


# Process each folder and .wav file
for folder_name in os.listdir(root_path):
    folder_path = os.path.join(root_path, folder_name)
    if os.path.isdir(folder_path):
        # Find all .wav files in the current folder
        for wav_file_path in glob.glob(os.path.join(folder_path, '*.wav')):
            # Read the analog audio signal (wav file)
            with wave.open(wav_file_path, 'r') as wav_file:
                n_channels = wav_file.getnchannels()
                sample_width = wav_file.getsampwidth()
                framerate = wav_file.getframerate()
                n_frames = wav_file.getnframes()
                audio_signal = wav_file.readframes(n_frames)

            # Convert audio signal to numpy array
            audio_signal = np.frombuffer(audio_signal, dtype=np.int16)

            # Pre-filtering
            filtered_signal = butter_bandpass_filter(audio_signal, lowcut, highcut, fs)

            # Sampling (downsampling if necessary)
            sampled_signal = filtered_signal[::int(framerate/fs)]

            # Quantization (assuming the signal is already 16-bit, no additional quantization is needed)
            quantized_signal = np.array(sampled_signal, dtype=np.int16)

            # Coding (compression) - This step would depend on the coding algorithm used
            # For example, applying a simple delta encoding (not an actual compression algorithm)
            coded_signal = np.diff(quantized_signal)

            # Extract the audio file name without extension
            audio_file_name = os.path.splitext(os.path.basename(wav_file_path))[0]

            # Output file path
            output_file_path = os.path.join(folder_path, f'preprocessed_{audio_file_name}.wav')

            # Save the digitized audio signal as a .wav file
            with wave.open(output_file_path, 'w') as wav_out:
                wav_out.setnchannels(n_channels)
                wav_out.setsampwidth(sample_width)
                wav_out.setframerate(fs)
                wav_out.writeframes(coded_signal.tobytes())

            print(f'Processed and saved: {output_file_path}')

Processed and saved: d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data\300_P\preprocessed_300_AUDIO.wav


Error: file does not start with RIFF id